In [2]:
import math
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from haversine import haversine, Unit

tqdm.pandas(desc='Progress')

In [31]:
def correct_rounding(number):
    if number > 0:
        return int(number + 0.5)
    else:
        return int(number + -0.5)
    
def create_coordinate_list (lat_matrix, lon_matrix):
    N, M = lat_matrix.shape
    coordinate_list = []
    for i in range(N):
        for j in range(M):
            coordinate_list.append([lat_matrix[i, j], lon_matrix[i, j]])
    return coordinate_list

def euclidean_distance(point1: list, point2: list):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))

def deg2rad(deg):
    return deg * (math.pi / 180.0)

def get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    r = 6371.0
    d_lat = deg2rad(lat2 - lat1)
    d_lon = deg2rad(lon2 - lon1)
    a = (math.sin(d_lat / 2.0) * math.sin(d_lat / 2.0) + math.cos(deg2rad(lat1))
         * math.cos(deg2rad(lat2)) * math.sin(d_lon / 2.0) * math.sin(d_lon / 2.0))
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))
    d = r * c
    return d

def find_closest_coordinates(point: list, coordinates_list: list):
    min_dist = float('inf')
    closest_coord = None
    for coords in coordinates_list:
        eucl_distances = get_distance_from_lat_lon_in_km(*point, *coords)
        if eucl_distances <= min_dist:
            min_dist = eucl_distances
            closest_coord = coords
    return closest_coord   


def find_index (lat_matrix, lon_matrix, target_coord):
    target_lat = target_coord[0]
    target_lon = target_coord[1]
    N, M = lat_matrix.shape
    result_index = None
    
    for i in range(N):
        for j in range(M):
            lat = lat_matrix[i, j]
            lon = lon_matrix[i, j]
            if lat == target_lat and lon == target_lon:
                result_index = i, j
    return result_index

def get_correct_lon(lon: int):
    if lon > 180:
        lon = lon - 360
    return lon

### Подгрузка данных

In [32]:
sheets = pd.ExcelFile('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx').sheet_names
lon = pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name='lon')
lon = lon.applymap(get_correct_lon)
lon = np.array(lon)
lat = np.array(pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name='lat'))
week_data_dict = {}
for sheet in tqdm(sheets[2:]):
    data = pd.read_excel('/home/kmulygin/khakaton/data/IntegrVelocity.xlsx', header=None, sheet_name=sheet)
    data = data.applymap(correct_rounding)
    # coordinates_integr_velocity = np.array([lat, lon, data])
    week_data_dict[sheet] = data
    
lat_lon_coordinates = create_coordinate_list(lat, lon)

/tmp/ipykernel_2889/476155299.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lon = lon.applymap(get_correct_lon)
  0%|          | 0/14 [00:00<?, ?it/s]/tmp/ipykernel_2889/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
  7%|▋         | 1/14 [00:00<00:04,  2.80it/s]/tmp/ipykernel_2889/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 14%|█▍        | 2/14 [00:00<00:04,  2.87it/s]/tmp/ipykernel_2889/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 21%|██▏       | 3/14 [00:01<00:07,  1.41it/s]/tmp/ipykernel_2889/476155299.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(correct_rounding)
 29%|██▊       | 4/14 [00:02<00:

In [33]:
reference_graph_points = pd.read_excel('/home/kmulygin/khakaton/data/ГрафДанные.xlsx', sheet_name = 'points')
reference_graph_edges = pd.read_excel('/home/kmulygin/khakaton/data/ГрафДанные.xlsx', sheet_name = 'edges')

### Нахождение необходимых координат

In [34]:
def get_graph_index(initial_coordinates: list):
    closest_coordinate = find_closest_coordinates(initial_coordinates, lat_lon_coordinates)
    return find_index(lat, lon, closest_coordinate)

In [35]:
dict_pointid_lat_lon = {}
for _, row in reference_graph_points.iterrows():
    dict_pointid_lat_lon[row['point_id']] = [[row['latitude'], row['longitude']], row['point_name']]

In [36]:
start_data = reference_graph_edges[['start_point_id', 'end_point_id', 'length']]
start_coordinates = []
end_coordinates = []
name_start = []
name_end = []
for _, row in start_data.iterrows():
    start_coordinates.append(dict_pointid_lat_lon[row['start_point_id']][0])
    name_start.append(dict_pointid_lat_lon[row['start_point_id']][1])
    end_coordinates.append(dict_pointid_lat_lon[row['end_point_id']][0])
    name_end.append(dict_pointid_lat_lon[row['end_point_id']][1])

start_data['start_coordinate'] = start_coordinates
start_data['end_coordinate'] = end_coordinates
start_data['name_start'] = name_start
start_data['name_end'] = name_end
start_data['graph_start_index'] = start_data['start_coordinate'].progress_apply(lambda x: get_graph_index(x))
start_data['graph_end_index'] = start_data['end_coordinate'].progress_apply(lambda x: get_graph_index(x))

/tmp/ipykernel_2889/977449582.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_data['start_coordinate'] = start_coordinates
/tmp/ipykernel_2889/977449582.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_data['end_coordinate'] = end_coordinates
/tmp/ipykernel_2889/977449582.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

### Построение графа и визуализация

In [ ]:
([{'ship_type': 'Arc 4', 'ship_name': 'Имя_4', 'max_speed': 16}], ice_matrix)

{'ship_type': {"14-10": * 0.8}}

In [8]:
# для ледоколов необходимо прописывать отдельное ограничение
def get_graph_without_icebreaker (speed_matrix: np.array, ship_type: str, ship_name: str, max_personal_speed: float) -> nx.DiGraph:    
    N, M= speed_matrix.shape
   
    G = nx.DiGraph()
    # Добавляем ребра и веса
    if ship_type == 'Нет':
        for i in range(N):
            for j in range(M):
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 4' or ship_type == 'Arc 5' or ship_type == 'Arc 6':
        for i in range(N):
            for j in range(M):
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 7':
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * speed_matrix[i+1, j], max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * speed_matrix[i+1, j], max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (max(start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if end_point >= 20 and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (end_point >= 15 and end_point <= 19) and (start_point >= 15 and start_point <= 19):
                        avg_speed = (max(0.6 * start_point, max_personal_speed) + max(0.6 * end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
    elif ship_type == 'Arc 9' and (ship_name == '50 лет Победы' or ship_name == 'Ямал'):
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j), weight=weight)
                            G.add_edge((i+1, j), (i, j), weight=weight)
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i, j + 1), weight=weight)
                            G.add_edge((i, j + 1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j+1), weight=weight)
                            G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point > 0 and end_point > 0:
                        if start_point >= 20 and end_point >= 20:
                            avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                        elif (start_point >= 10 and start_point < 20) and end_point >= 20:
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
                        elif start_point >= 20 and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
                        elif (start_point >= 10 and start_point < 20) and (end_point >= 10 and end_point < 20):
                            avg_speed = (start_point + end_point) / 2
                            dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                            weight = dist / avg_speed
                            G.add_edge((i, j), (i+1, j-1), weight=weight)
                            G.add_edge((i+1, j-1), (i, j), weight=weight)
                            
    elif  ship_type == 'Arc 9' and (ship_name == 'Вайгач' or ship_name == 'Таймыр'):
        for i in range(N):
            for j in range(M):
                
                if i < N-1:  # вертикальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1, j])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j], lon[i+1, j]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j), weight=weight)
                        G.add_edge((i+1, j), (i, j), weight=weight)
                        
                        
                        
                        
                if j < M-1:  # горизонтальные ребра
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i, j + 1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i, j+1], lon[i, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i, j+1), weight=weight)
                        G.add_edge((i, j+1), (i, j), weight=weight)
                        
                # if i < N-1 and j < N-1:  # диагональные ребра вниз вправо
                if i < N-1 and j < M-1:  # диагональные ребра вниз вправо
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j+1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j+1], lon[i+1, j+1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j+1), weight=weight)
                        G.add_edge((i+1, j+1), (i, j), weight=weight)
                        
                        
                        
                # if i < M-1 and j > 0:  # диагональные ребра вниз влево
                if i < N-1 and j > 0:  # диагональные ребра вниз влево
                    
                    start_point = correct_rounding(speed_matrix[i, j])
                    end_point = correct_rounding(speed_matrix[i+1 , j-1])
                    
                    if start_point >= 20 and end_point >= 20:
                        avg_speed = (max(start_point, max_personal_speed) + max(end_point, max_personal_speed)) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 15 and end_point <= 19):
                        avg_speed = (start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if start_point >= 20 and (end_point >= 10 and end_point <= 14):
                        avg_speed = (start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and end_point >= 20:
                        avg_speed = (0.9 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.9 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 15 and start_point <= 19) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.9 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and end_point >= 20:
                        avg_speed = (0.75 * start_point + end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 15 and end_point <= 19):
                        avg_speed = (0.75 * start_point + 0.9 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                        
                    if (start_point >= 10 and start_point <= 14) and (end_point >= 10 and end_point <= 14):
                        avg_speed = (0.75 * start_point + 0.75 * end_point) / 2
                        dist = haversine((lat[i, j], lon[i, j]), (lat[i+1, j-1], lon[i+1, j-1]), unit='mi')
                        weight = dist / avg_speed
                        G.add_edge((i, j), (i+1, j-1), weight=weight)
                        G.add_edge((i+1, j-1), (i, j), weight=weight)
                
    return G


In [9]:
def graph_visualize(speed_matrix: np.array, graph: nx.classes.digraph.DiGraph):
    
    N, M = speed_matrix.shape
    pos = {(i, j): (j, -i) for i in range(N) for j in range(M)}  # расположение узлов для визуализации
    labels = {node: f"{node}\n{speed_matrix[node[0], node[1]]} км/ч" for node in graph.nodes()}
    edge_labels = {(u, v): f"{d['weight']:.2f} ч" for u, v, d in graph.edges(data=True)}

    print('Отрисовка графика начата')
    plt.figure(figsize=(6, 6))
    nx.draw(graph, pos, with_labels=True, labels=labels, node_size=2000, node_color="lightblue", font_size=10, font_color="black", font_weight="bold")
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_color='red')
    plt.title("Граф возможных перемещений с весами ребер (время в часах)")
    plt.show()

### Аглоритм Дейкстры

In [18]:
def get_path(graph, index_start, index_end):
    try:
        _, path = nx.single_source_dijkstra(graph, index_start, index_end, weight='weight')
        return path
    except:
        return 'Нужен ледокол'

def get_cost(graph, index_start, index_end):
    try:
        cost, _ = nx.single_source_dijkstra(graph, index_start, index_end, weight='weight')
        return cost
    except:
        return 'Нужен ледокол'

### Тестирование

In [12]:
ship_types = ['Нет', 'Arc 4', 'Arc 7', 'Arc 9', 'Arc 9']
ship_names = ['Имя_судна_1', 'Имя_судна_2', 'Имя_судня_3', 'Ямал', 'Вайгач']
ship_speeds = [14, 19, 19, 21, 18.5]

In [13]:
result_model_output = {}
for week in tqdm(week_data_dict):
    result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        result_model_output[week][sheep_types_name] = {}
        result_model_output[week][sheep_types_name]['path'] = {}
        result_model_output[week][sheep_types_name]['costs'] = {}
        Graph = get_graph_without_icebreaker(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])
        # Graph_data = nx.node_link_data(Graph)
        # with open(f"graphs_denis/{week}_{sheep_types_name}_graph.json", "w") as f:
        #     json.dump(Graph_data, f)
        for _, row in start_data.iterrows():
            result_model_output[week][sheep_types_name]['path'][f"{row['name_start']} + {row['name_end']}"] = get_path(Graph, row['graph_start_index'], row['graph_end_index'])
            result_model_output[week][sheep_types_name]['costs'][f"{row['name_start']} + {row['name_end']}"] = get_cost(Graph, row['graph_start_index'], row['graph_end_index'])
            

100%|██████████| 14/14 [02:12<00:00,  9.44s/it]


### Решение проблем с координатами некоторых портов для ледокола Ямал

In [14]:
bad_ports = {}
for week in list(week_data_dict.keys()):
    bad_ports[week] = []
    for key, values in result_model_output[week]['Arc 9_Ямал']['costs'].items():
        if values == 'Нужен ледокол':
            bad_ports[week].append(key)
    bad_ports[week] = list(set(bad_ports[week]))

In [15]:
list_bad_routes = []
compare = bad_ports['03-Mar-2020']
for week in bad_ports:
    for route in bad_ports[week]:
        list_bad_routes.append(route)
        
list_bad_ports = []
for ports in list_bad_routes:
    ports = ports.split(' + ')
    for port in ports:
        list_bad_ports.append(port)
        
list_bad_ports = list(set(list_bad_ports))

dict_bad_ports_coordinates = {}
for port in list_bad_ports:
    for _, row in start_data.iterrows():
        if row['name_start'] == port:
            dict_bad_ports_coordinates[port] = row['graph_start_index']
        elif row['name_end'] == port:
            dict_bad_ports_coordinates[port] = row['graph_end_index']

In [16]:
all_date_coord_list = {}

for week in week_data_dict:
    N, M = week_data_dict[week].shape
    matrix = np.array(week_data_dict[week])
    coord_list = []
    for i in range(N):
        for j in range(M):
            if matrix[i, j] >= 10:
                coord_list.append((i, j))
    all_date_coord_list[week] = coord_list

In [17]:
target_port = 'Индига'
target_coordinate = (27, 48)
shortest_point_for_bad_ports  = {}

for week in tqdm(list(week_data_dict.keys())):
    coord_list = all_date_coord_list[week]
    shortest_point_for_bad_ports[week] = {}
    Graph = get_graph_without_icebreaker(speed_matrix=np.array(week_data_dict[week]), max_personal_speed=21, ship_name='Ямал', ship_type='Arc 9')
    # with open('khakaton/shortest_point_for_bad_ports.pkl', 'wb') as f:
    #     pickle.dump(shortest_point_for_bad_ports, f)
    for port in dict_bad_ports_coordinates:
        min_distance = float('inf')
        path = get_path(graph=Graph, index_start=target_coordinate, index_end=dict_bad_ports_coordinates[port])
        if path == 'Нужен ледокол':
            for coord in coord_list:
                distance = haversine((lat[coord[0], coord[1]], lon[coord[0], coord[1]]), (lat[dict_bad_ports_coordinates[port][0], dict_bad_ports_coordinates[port][1]], lon[dict_bad_ports_coordinates[port][0], dict_bad_ports_coordinates[port][1]]), unit='mi')
                if distance < min_distance:
                    min_distance = distance
                    shortest_point_for_bad_ports[week][port] = coord
        
        else:
            continue
    

100%|██████████| 14/14 [00:35<00:00,  2.54s/it]


### Корректировка ошибки алгоритма (удаление лишних новых координат портов)

In [18]:
target_port = 'Индига'
target_coordinate = (27, 48)
ports_not_for_new_coordinates  = {}

for week in tqdm(list(week_data_dict.keys())):
    ports_not_for_new_coordinates[week] = []
    Graph = get_graph_without_icebreaker(speed_matrix=np.array(week_data_dict[week]), max_personal_speed=21, ship_name='Ямал', ship_type='Arc 9')
    
    for port in dict_bad_ports_coordinates:
        min_distance = float('inf')
        path = get_path(graph=Graph, index_start=target_coordinate, index_end=dict_bad_ports_coordinates[port])
        if path != 'Нужен ледокол':
            ports_not_for_new_coordinates[week].append(port)

100%|██████████| 14/14 [00:32<00:00,  2.32s/it]


In [19]:
correct_shortest_point_for_bad_ports = {}
for week in shortest_point_for_bad_ports:
    correct_shortest_point_for_bad_ports[week] = {}
    for port in shortest_point_for_bad_ports[week]:
        if port not in ports_not_for_new_coordinates[week]:
            correct_shortest_point_for_bad_ports[week][port] = shortest_point_for_bad_ports[week][port]

In [20]:
def replace_bad_port_coordinates(row):
    for port in correct_shortest_point_for_bad_ports[week]:
        if row['name_start'] == port:
            row['graph_start_index'] = correct_shortest_point_for_bad_ports[week][port]
        if row['name_end'] == port:
            row['graph_end_index'] = correct_shortest_point_for_bad_ports[week][port]
    return row

In [21]:
dataset_with_normal_coordinates = {}
for week in list(correct_shortest_point_for_bad_ports.keys()):
    new_start_data =  start_data.apply(replace_bad_port_coordinates, axis=1)
    dataset_with_normal_coordinates[week] = new_start_data

### Построение новых маршрутов с верными координатами

In [21]:
result_model_output = {}
for week in tqdm(week_data_dict):
    result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        result_model_output[week][sheep_types_name] = {}
        result_model_output[week][sheep_types_name]['path'] = {}
        result_model_output[week][sheep_types_name]['costs'] = {}
        Graph = get_graph_without_icebreaker(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])
        # Graph_data = nx.node_link_data(Graph)
        # with open(f"graphs_denis/{week}_{sheep_types_name}_graph.json", "w") as f:
        #     json.dump(Graph_data, f)
        for _, row in dataset_with_normal_coordinates[week].iterrows():
            result_model_output[week][sheep_types_name]['path'][f"{row['name_start']} + {row['name_end']}"] = get_path(Graph, row['graph_start_index'], row['graph_end_index'])
            result_model_output[week][sheep_types_name]['costs'][f"{row['name_start']} + {row['name_end']}"] = get_cost(Graph, row['graph_start_index'], row['graph_end_index'])
            

100%|██████████| 14/14 [01:43<00:00,  7.41s/it]


In [22]:
dataset_with_normal_coordinates['03-Mar-2020'][dataset_with_normal_coordinates['03-Mar-2020']['name_start'] == 'Индига']

,start_point_id,end_point_id,length,start_coordinate,end_coordinate,name_start,name_end,graph_start_index,graph_end_index
0,44,15,270.016642,"[67.58, 47.82]","[70.3, 57.8]",Индига,Карские ворота,"(27, 48)","(47, 55)"
61,44,2,156.064482,"[67.58, 47.82]","[69.9, 44.6]",Индига,кромка льда на Западе,"(27, 48)","(35, 39)"


In [23]:
result_model_output['03-Mar-2020']['Arc 9_Ямал']['costs']

{'Индига + Карские ворота': 16.763489000292473,
 'Вход в Обскую губу + Новый порт': 24.798429756985644,
 'Восточно-Сибирское - 1 (восток) + Рейд Певек': 5.17142688069621,
 'Карское - 1 (сбор каравана) + Мыс Желания': 17.468863907463124,
 'Вход в Обскую губу + Карское - 1 (сбор каравана)': 6.0864441546993735,
 'около Новой Земли + Окно в Европу': 32.842953702601086,
 'Штокман + Окно в Европу': 21.39879473480397,
 'Берингов пролив + Берингово': 15.255383898539861,
 'Карское - 1 (сбор каравана) + Карское - 3 (центр)': 12.891200051772817,
 'Лаптевых - 4 (юг) + устье Лены': 11.536798577208835,
 'пролив Лонга + Восточно-Сибирское - 1 (восток)': 15.186335549716928,
 'Восточно-Сибирское - 3 (север) + Восточно-Сибирское - 2 (запад)': 23.968018676108695,
 'Лаптевых - 2 (центр) + Пролив Санникова - 1': 18.49833088645622,
 'остров Котельный + Лаптевых - 3 (восток)': 12.56476097986969,
 'МОТ Печора + Варандей-Приразломное': 4.91423815121096,
 'Бухта Север и Диксон + Карское - 2 (прибрежный)': 7.376

### Корректировка выдачи под формат Саши и Аси

In [64]:
new_result_model_output = {}
for week in tqdm(week_data_dict):
    new_result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    dataset_norm_coordinates = dataset_with_normal_coordinates[week]
    
    required_data_nodes = {}
    for _, row in dataset_norm_coordinates.iterrows():
        if row['start_point_id'] not in required_data_nodes:
            required_data_nodes[row['start_point_id']] = [row['start_point_id'], row['name_start'], row['start_coordinate'], row['graph_start_index']]
        if row['end_point_id'] not in required_data_nodes:
            required_data_nodes[row['end_point_id']] = [row['end_point_id'], row['name_end'], row['end_coordinate'], row['graph_end_index']]
        
    required_data_edges = {}
    for idx, row in dataset_norm_coordinates.iterrows():
        required_data_edges[idx] = [row['start_point_id'], row['end_point_id'], row['graph_start_index'], row['graph_end_index']]
        
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        new_result_model_output[week][sheep_types_name] = {}
        Graph = get_graph_without_icebreaker(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])
        new_result_model_output[week][sheep_types_name]['nodes'] = [] 
        for id in required_data_nodes:
            new_result_model_output[week][sheep_types_name]['nodes'].append({'id': required_data_nodes[id][0], 'name': required_data_nodes[id][1],
                                                                             'coordinates': [lat[required_data_nodes[id][3][0], required_data_nodes[id][3][1]],
                                                                                             lon[required_data_nodes[id][3][0], required_data_nodes[id][3][1]]]})
            
        new_result_model_output[week][sheep_types_name]['edges'] = []
        
        for idx in required_data_edges:
            new_result_model_output[week][sheep_types_name]['edges'].append({'target': required_data_edges[idx][0], 'source':  required_data_edges[idx][1],
                                                                         'path': get_path(Graph, required_data_edges[idx][2], required_data_edges[idx][3]), 
                                                                         'costs': get_cost(Graph, required_data_edges[idx][2], required_data_edges[idx][3])})
        
            
            

            

100%|██████████| 14/14 [01:45<00:00,  7.52s/it]


In [ ]:
# import pickle
# with open('/home/kmulygin/khakaton/new_format_result.pkl', 'wb') as fp:
#     pickle.dump(new_result_model_output, fp)

### Тестирование алгоритма для определения кратчайших путей до ледового графа

In [22]:
# формирование подходящего формата выхода
new_result_model_output = {}
for week in tqdm(week_data_dict):
    new_result_model_output[week] = {}
    speed_matrix = np.array(week_data_dict[week])
    dataset_norm_coordinates = dataset_with_normal_coordinates[week]
    
    required_data_nodes = {}
    for _, row in dataset_norm_coordinates.iterrows():
        if row['start_point_id'] not in required_data_nodes:
            required_data_nodes[row['start_point_id']] = [row['start_point_id'], row['name_start'], row['start_coordinate'], row['graph_start_index']]
        if row['end_point_id'] not in required_data_nodes:
            required_data_nodes[row['end_point_id']] = [row['end_point_id'], row['name_end'], row['end_coordinate'], row['graph_end_index']]
        
    required_data_edges = {}
    for idx, row in dataset_norm_coordinates.iterrows():
        required_data_edges[idx] = [row['start_point_id'], row['end_point_id'], row['graph_start_index'], row['graph_end_index']]
        
    for i in range(len(ship_types)):
        sheep_types_name = f"{ship_types[i]}_{ship_names[i]}"
        new_result_model_output[week][sheep_types_name] = {}
        Graph = get_graph_without_icebreaker(speed_matrix=speed_matrix, ship_type=ship_types[i], ship_name=ship_names[i], max_personal_speed=ship_speeds[i])
        new_result_model_output[week][sheep_types_name]['nodes'] = [] 
        for id in required_data_nodes:
            new_result_model_output[week][sheep_types_name]['nodes'].append({'id': required_data_nodes[id][0], 'name': required_data_nodes[id][1],
                                                                             'coordinates': [lat[required_data_nodes[id][3][0], required_data_nodes[id][3][1]],
                                                                                             lon[required_data_nodes[id][3][0], required_data_nodes[id][3][1]]],
                                                                             'graph_coordinates': required_data_nodes[id][3]})
            
        new_result_model_output[week][sheep_types_name]['edges'] = []
        
        for idx in required_data_edges:
            new_result_model_output[week][sheep_types_name]['edges'].append({'target': required_data_edges[idx][0], 'source':  required_data_edges[idx][1],
                                                                         'path': get_path(Graph, required_data_edges[idx][2], required_data_edges[idx][3]), 
                                                                         'costs': get_cost(Graph, required_data_edges[idx][2], required_data_edges[idx][3])})
        
            
            

            

100%|██████████| 14/14 [01:40<00:00,  7.17s/it]


In [23]:
with open('/home/kmulygin/khakaton/ice_graph_boundary_points.pkl', 'rb') as f:
    boundary_points = pickle.load(f)

In [24]:
def get_closest_boundary_points(list_of_boundary_points: list, graph: nx.DiGraph, start_index: tuple) -> tuple:
    min_cost = float('inf')
    closest_index = None
    if start_index in graph.nodes():
        for point in list_of_boundary_points:
            cost = get_cost(graph, start_index, point)
            if cost == 'Нужен ледокол':
                continue
            else:
                if cost < min_cost:
                    min_cost = cost
                    closest_index = point
        return closest_index, min_cost
    else:
        return start_index, 0

In [25]:
ship_types = ['Нет', 'Arc 4', 'Arc 7', 'Arc 9', 'Arc 9']
ship_names = ['Имя_судна_1', 'Имя_судна_2', 'Имя_судня_3', 'Ямал', 'Вайгач']
ship_speeds = [14, 19, 19, 21, 18.5]
sheep_types_name_speed_dict = {'Нет_Имя_судна_1': ['Нет', 'Имя_судна_1', 14],
                               'Arc 4_Имя_судна_2': ['Arc 4', 'Имя_судна_2', 19],
                               'Arc 7_Имя_судня_3': ['Arc 7', 'Имя_судня_3', 19]}

path_to_boundary_points = {}

for week in tqdm(new_result_model_output):
    path_to_boundary_points[week] = {}
    for sheep in tqdm(new_result_model_output[week]):
        if sheep != 'Arc 9_Ямал' and sheep != 'Arc 9_Вайгач':
            Graph = get_graph_without_icebreaker(np.array(week_data_dict[week]), sheep_types_name_speed_dict[sheep][0], sheep_types_name_speed_dict[sheep][1], 
                                                 sheep_types_name_speed_dict[sheep][2])
            path_to_boundary_points[week][sheep] = {}
            path_to_boundary_points[week][sheep]['start_points'] = []
            path_to_boundary_points[week][sheep]['end_points'] = []
            for item in new_result_model_output[week][sheep]:
                for edge in new_result_model_output[week][sheep]['edges']:
                    if edge['path'] == 'Нужен ледокол':
                        for dicts in new_result_model_output[week][sheep]['nodes']:
                            if dicts['id'] == edge['target']:
                                start_point_id = dicts['id']
                                start_point_name = dicts['name']
                                start_point_coordinates = dicts['coordinates']
                                start_point_graph_coordinates = dicts['graph_coordinates']
                            if dicts['id'] == edge['source']:
                                end_point_id = dicts['id']
                                end_point_name = dicts['name']
                                end_point_coordinates = dicts['coordinates']
                                end_point_graph_coordinates = dicts['graph_coordinates']
    
                        closest_point_to_start, start_cost = get_closest_boundary_points(boundary_points[week][sheep], Graph, start_point_graph_coordinates)
                        path_to_boundary_points[week][sheep]['start_points'].append({'id': start_point_id, 'point_name': start_point_name,
                                                                                        'coordinates': start_point_coordinates, 'graph_index': start_point_graph_coordinates,
                                                                                        'closest_ice_graph_index': closest_point_to_start, 'cost': start_cost})
                       
                        closest_point_to_end, end_cost = get_closest_boundary_points(boundary_points[week][sheep], Graph, end_point_graph_coordinates)
                        path_to_boundary_points[week][sheep]['end_points'].append({'id': end_point_id, 'point_name': end_point_name,
                                                                                        'coordinates': end_point_coordinates, 'graph_index': end_point_graph_coordinates,
                                                                                        'closest_ice_graph_index': closest_point_to_end, 'cost': end_cost})
                            
                            
                            

100%|██████████| 14/14 [42:35<00:00, 182.54s/it]


In [94]:
# with open('/home/kmulygin/khakaton/intermediate_path_to_boundary_points.pkl', 'wb') as f:
# 	pickle.dump(path_to_boundary_points, f)

with open('/home/kmulygin/khakaton/intermediate_path_to_boundary_points.pkl', 'rb') as f:
	path_to_boundary_points = pickle.load(f)

In [95]:
for i in path_to_boundary_points['03-Mar-2020']['Нет_Имя_судна_1']:
    for ids in  path_to_boundary_points['03-Mar-2020']['Нет_Имя_судна_1'][i]:
        if ids['id'] == 42:
            print(ids, i)

{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} start_points
{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} start_points
{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} end_points
{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} end_points
{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} end_points
{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0} end_points


In [96]:
new_path_to_boundary_points = {}
for week in path_to_boundary_points:
    new_path_to_boundary_points[week] = {}
    for sheep in path_to_boundary_points[week]:
        new_path_to_boundary_points[week][sheep] = []
        list_of_id = []
        for type in path_to_boundary_points[week][sheep]:
            for item in path_to_boundary_points[week][sheep][type]:
                if item['id'] not in list_of_id:
                    list_of_id.append(item['id'])
                    new_path_to_boundary_points[week][sheep].append(item)
                else:
                    continue                        

In [97]:
for i in new_path_to_boundary_points['03-Mar-2020']['Нет_Имя_судна_1']:
    if i['id'] == 42:
        print(i)

{'id': 42, 'point_name': 'остров Котельный', 'coordinates': [76.6424, 141], 'graph_index': (147, 51), 'closest_ice_graph_index': (147, 51), 'cost': 0}


In [98]:
# with open('/home/kmulygin/khakaton/result_path_to_boundary_points.pkl', 'wb') as f:
# 	pickle.dump(new_path_to_boundary_points, f)

In [52]:
with open('/home/kmulygin/khakaton/result_path_to_boundary_points.pkl', 'rb') as f:
	path_to_boundary_points = pickle.load(f)
 
with open('/home/kmulygin/khakaton/ice_graphs.pkl', 'rb') as f:
	ice_graphs = pickle.load(f)

In [59]:
G_1 = nx.compose(ice_graphs['03-Mar-2020']['Нет_Имя_судна_1'], ice_graphs['03-Mar-2020']['Arc 4_Имя_судна_2'])
G_2 = nx.compose(G_1, ice_graphs['03-Mar-2020']['Arc 7_Имя_судня_3'])

In [60]:
get_path(G_2, (201, 31), (65, 67))

[(201, 31),
 (200, 32),
 (199, 32),
 (198, 33),
 (197, 32),
 (196, 32),
 (195, 32),
 (194, 33),
 (193, 33),
 (192, 33),
 (191, 33),
 (190, 33),
 (189, 34),
 (188, 34),
 (187, 34),
 (186, 34),
 (185, 34),
 (184, 34),
 (183, 34),
 (182, 34),
 (181, 34),
 (180, 34),
 (179, 34),
 (178, 34),
 (177, 34),
 (176, 34),
 (175, 34),
 (174, 34),
 (173, 34),
 (172, 34),
 (171, 35),
 (170, 36),
 (169, 36),
 (168, 37),
 (167, 38),
 (166, 38),
 (165, 39),
 (164, 39),
 (163, 40),
 (162, 40),
 (161, 41),
 (160, 42),
 (159, 42),
 (158, 43),
 (157, 43),
 (156, 43),
 (155, 44),
 (154, 44),
 (153, 45),
 (152, 46),
 (151, 46),
 (150, 46),
 (149, 47),
 (148, 47),
 (147, 47),
 (146, 48),
 (145, 48),
 (144, 48),
 (143, 48),
 (142, 48),
 (141, 48),
 (140, 48),
 (139, 48),
 (138, 48),
 (137, 48),
 (136, 48),
 (135, 48),
 (134, 48),
 (133, 48),
 (132, 48),
 (131, 48),
 (130, 48),
 (129, 48),
 (128, 48),
 (127, 48),
 (126, 48),
 (125, 48),
 (124, 48),
 (123, 48),
 (122, 48),
 (121, 48),
 (120, 48),
 (119, 48),
 (11

In [48]:
# get_path(ice_graphs['19-May-2020']['Нет_Имя_судна_1'], (201, 31), (65, 67))
# nx.single_source_dijkstra(GGt, (201, 31), (65, 67))